In [1]:
import numpy as np

In [2]:
with open("contribution_node2vec_out.txt") as f:
    dim_data = f.readline().split()
    num_nodes = int(dim_data[0])
    node_vec_dim = int(dim_data[1])
    node_vecs = np.zeros((int(dim_data[0]), int(dim_data[1])))
    for line in f:
        node_data = line.split()
        node_id = int(node_data[0])
        for i in range(node_vec_dim):
            node_vecs[node_id][i] = float(node_data[i+1])

In [3]:
print node_vecs[5987, :]

[ 0.406381  0.045738  0.41004  -0.025185 -0.178681 -0.697899 -0.061132
 -0.012406  0.258448  0.046634  0.384956  0.099652 -0.278242 -0.197552
  0.111555 -0.246045  0.222929  0.317824 -0.135143  0.276308  0.003248
  0.519326 -0.285762 -0.162088  0.04991   0.334705  0.001779 -0.030048
 -0.001241  0.210782 -0.037096 -0.00095   0.118053 -0.39036  -0.25364
 -0.317647 -0.046374 -0.401751 -0.28985  -0.369068 -0.010145  0.03013
  0.412739 -0.444577 -0.569927  0.37159   0.110973  0.455195  0.166442
 -0.236721  0.202703 -0.112102  0.293576 -0.509746  0.163716  0.029895
  0.275661 -0.007347 -0.835764  0.273082  0.244738 -0.163165  0.014589
 -0.757269  0.004669 -0.1448   -0.688509  0.070206  0.185529 -0.344874
 -0.120837 -0.421547 -0.49137  -0.168105  0.08401   0.327175  0.001633
 -0.319605  0.095285  0.128336 -0.010048  0.765562  0.616104 -0.372242
 -0.231031  0.025448  0.125265 -0.167782  0.221827 -0.164901 -0.364121
  0.752501 -0.289592  0.193644  0.311914  0.223319 -0.168007  0.116212
 -0.0995

In [4]:
from sklearn.cluster import KMeans

In [5]:
kmeans = KMeans(n_clusters=2)
labels = kmeans.fit_predict(node_vecs)

In [6]:
print np.unique(labels, return_counts=True)

(array([0, 1], dtype=int32), array([8266, 6888]))


In [7]:
print "===== Democrat politicians ====="
print "Nancy Pelosi (D):", labels[13625]
print "Bernie Sanders (D):", labels[6852]
print "Elizabeth Warren (D):", labels[1606]
print "Debbie Wasserman Schultz (D):", labels[2757]
print "Tim Kaine (D):", labels[4951]
print "Kirsten Gillibrand (D):", labels[3414]
print "Tim Ryan (D):", labels[12784]
print

print "===== Republican politicians ====="
print "Mitch McConnell (R):", labels[2955]
print "Paul Ryan (R):", labels[5959]
print "Michael McCaul (R):", labels[5750]
print "Marco Rubio (R):", labels[3139]
print "Rand Paul (R):", labels[578]
print

#Indices are invalid. Do not use.
print "===== Companies ======"
print "Goldman Sachs:", labels[2339]
print "Costco:", labels[3312]
print "University of Michigan:", labels[4025]
print "Libery mutual:", labels[4065]
print "Michigan Farm Bureau:", labels[4073]
print "National Tank Truck Carriers Inc", labels[4075]
print "Boeing co", labels[4081]
print "HR Trading LLC", labels[11481]
print "Mckinsey & co", labels[11545]
print "Harvard University", labels[1669]

===== Democrat politicians =====
Nancy Pelosi (D): 1
Bernie Sanders (D): 1
Elizabeth Warren (D): 1
Debbie Wasserman Schultz (D): 1
Tim Kaine (D): 1
Kirsten Gillibrand (D): 1
Tim Ryan (D): 1

===== Republican politicians =====
Mitch McConnell (R): 0
Paul Ryan (R): 0
Michael McCaul (R): 0
Marco Rubio (R): 0
Rand Paul (R): 0

===== Companies ======
Goldman Sachs: 1
Costco: 1
University of Michigan: 1
Libery mutual: 1
Michigan Farm Bureau: 0
National Tank Truck Carriers Inc 0
Boeing co 0
HR Trading LLC 0
Mckinsey & co 1
Harvard University 1


In [8]:
democrats = set()
republicans = set()
id_to_politician = {}
party_0 = []
party_1 = []

with open("contribution_id_map.txt") as f:
    for line in f:
        data = line.rstrip("\n").split(",")
        # politician
        if len(data) > 2:
            politician_id = int(data[0])
            politician_name = data[1]
            politician_party = data[3]
            id_to_politician[politician_id] = politician_name
            if politician_party == "democrat":
                democrats.add(politician_id)
                if labels[politician_id] == 0:
                    party_0.append(politician_id)
                else:
                    party_1.append(politician_id)
            elif politician_party == "republican":
                republicans.add(politician_id)
                if labels[politician_id] == 0:
                    party_0.append(politician_id)
                else:
                    party_1.append(politician_id)
        # company
        else:
            pass

In [9]:
party0_democrats = 0
party0_republicans = 0
for pol_id in party_0:
    if pol_id in democrats:
        party0_democrats += 1
    else:
        party0_republicans += 1
print "Party 0:", party0_democrats, party0_republicans
party0_total = party0_democrats + party0_republicans
party0_probability_democrat = float(party0_democrats)/party0_total
party0_probability_republican = float(party0_republicans)/party0_total
print "Party 0 entropy:", -(party0_probability_democrat * np.log2(party0_probability_democrat)
                            + party0_probability_republican * np.log2(party0_probability_republican))

party1_democrats = 0
party1_republicans = 0
for pol_id in party_1:
    if pol_id in democrats:
        party1_democrats += 1
    else:
        party1_republicans += 1
print "Party 1:", party1_democrats, party1_republicans
party1_total = party1_democrats + party1_republicans
party1_probability_democrat = float(party1_democrats)/party1_total
party1_probability_republican = float(party1_republicans)/party1_total
print "Party 1 entropy:", -(party1_probability_democrat * np.log2(party1_probability_democrat) +
                            party1_probability_republican * np.log2(party1_probability_republican))


Party 0: 12 274
Party 0 entropy: 0.251198816709
Party 1: 247 6
Party 1 entropy: 0.161821712221


In [10]:
party0_outlier = "democrat" if party0_democrats < party0_republicans else "republican"
if party0_outlier == "democrat":
    for politician_id in party_0:
        if politician_id in democrats:
            print id_to_politician[politician_id], "is a democrat outlier classified as republican."
    for politician_id in party_1:
        if politician_id in republicans:
            print id_to_politician[politician_id], "is a republican outlier classified as democrat."
elif party0_outlier == "republican":
    for politician_id in party_0:
        if politician_id in republicans:
            print id_to_politician[politician_id], "is a republican outlier classified as democrat."
    for politician_id in party_1:
        if politician_id in democrats:
            print id_to_politician[politician_id], "is a democrat outlier classified as republican."

john carney is a democrat outlier classified as republican.
sanford bishop is a democrat outlier classified as republican.
jim costa is a democrat outlier classified as republican.
tim johnson is a democrat outlier classified as republican.
ron kind is a democrat outlier classified as republican.
jim matheson is a democrat outlier classified as republican.
collin peterson is a democrat outlier classified as republican.
terri a. sewell is a democrat outlier classified as republican.
jim cooper is a democrat outlier classified as republican.
gregory w. meeks is a democrat outlier classified as republican.
g. k. butterfield is a democrat outlier classified as republican.
henry cuellar is a democrat outlier classified as republican.
ileana ros-lehtinen is a republican outlier classified as democrat.
tom petri is a republican outlier classified as democrat.
michael grimm is a republican outlier classified as democrat.
frank a. lobiondo is a republican outlier classified as democrat.
pete ki